# Optimization tutorial : Sudoku 

**KERROUMI Mohamed**

May 2019



In this tutorial we will use Python. The objective is to solve the puzzle Sudoku for any size (order), using integer programming.

## 1 - Fromulation
In this section, we will present the formulation of the IP problem of a Sudoku puzzle of size 3, the formulation is roughly the same for any size.
- The Sudoku puzzles are feasibility problems, we are only looking for a feasible solution and there's no objective funtion we are trying to maximize, so in our formulation we'll set the objective function to 0.
- To specify that a given number k is located or not in position $(i,j)$ in the grid, we introduce a binary variable $x_{kij}$ such as: $x_{kij}=1$ if k is located in position $(i,j)$, $x_{kij}=0$ otherwise.
- In filling the grid, each row, column, and three-by-three subgrid must contain only one of an integer value. so,
<br>
Line Constraints:
$$ \sum_{j=1}^9 x_{kij} =1 \qquad for \quad i,k \in \{1...9\}$$
Column Constraints:
$$ \sum_{i=1}^9 x_{kij} =1 \qquad for \quad j,k \in \{1...9\}$$
Subgrid Constraints:
$$ \sum_{i=1}^3 \sum_{j=1}^3 x_{k, 3p+i, 3q+j} =1 \qquad for \qquad q,p \in \{0,1,2\} \quad k \in \{1...9\}$$
Only a single number can be located at any location $(i,j)$:
$$ \sum_{k=1}^9 x_{kij} =1 \qquad for \qquad i,j \in \{1...9\}$$
Additional constraints of the known numbers:
$$x_{kij}=1 \qquad for \qquad i,j,k \in G = \text{All the known cells} $$
Hence, the formulation of the problem is as follows:
\begin{alignat}{4}
&   \max_{x}       &            &       & z = 0  && && \\
&\text{subject to} & \qquad     &             &\sum_{j=1}^9  & x_{kij}         &=&1 &\quad for \quad i,k \in \{1...9\}& \\
&                  &            &             &\sum_{i=1}^9 & x_{kij}          &=&1 &\quad for \quad j,k \in \{1...9\}& \\
&                  &            & \sum_{i=1}^3&\sum_{j=1}^3  &x_{k, 3p+i, 3q+j}&=&1 &\quad for \quad q,p\in \{0,1,2\}&\quad k \in \{1...9\}\\
&                  &            &             &\sum_{k=1}^9 & x_{kij}          &=&1 &\quad for \quad i,j \in \{1...9\}& \\
&                  &            &             & & x_{kij}          &=&1 &\quad for \quad i,j,k \in G& \\
\end{alignat}

## 2 - Resolution using Python

In [1]:
import numpy as np

import cvxopt

import cvxopt.glpk
cvxopt.glpk.ilp?

from cvxopt import matrix

In [84]:
# The variables of our problem are well understood if they are represented in 3D matrix, so we'll write  two functions to go
#from the 3D representation to 1D, and vice versa:
def to1D(i,j,k,size=3):
    n=size**2
    assert( 0<=i and i<n)
    assert( 0<=j and j<n)
    assert( 0<=k and k<n)
    return(j+n*i+k* (n)**2)
def to3D(l, size):
    n=size**2
    assert( 0<=l and l<n**3)
    k= l//(n**2)
    i= (l-k*(n)**2)//n
    j= l -n*i -k*(n)**2
    return (i,j,k)

In [85]:
#The function that creates the matrix of constraints A
def const(B,size=3):
    n=size**2
    A=np.zeros((4*n*n,n*n*n))
    #contraint: Only a single number can be located at any location (i,j)
    for i in range(n):
        for j in range(n):
            for k in range(n):
                A[i+ j*n,to1D(i,j,k,size)]=1
    #The line contraints:
    for k in range(n):
        for i in range(n):
            for j in range(n):
                A[n**2 + k+ i*n,to1D(i,j,k,size)]=1
    #The column contraints:
    for k in range(n):
        for j in range(n):
            for i in range(n):
                A[2*(n**2) + k+ j*n,to1D(i,j,k,size)]=1
    #Square constraints:
    for k in range(n):
        for p in range(size):
            for q in range(size):
                for i in range(size):
                    for j in range(size):
                        A[3*(n**2)+p+q*size+k*n,to1D(size*p+i,size*q+j,k,size)]=1
    for m in range(np.shape(A)[0]):
        assert(sum(A[m,])==n)
    #Additional constraints of the known numbers
    for k in range(1,n+1):
        result=np.where(B==k)
        # zip the 2 arrays to get the exact coordinates
        listOfCoordinates= list(zip(result[0], result[1]))
        for e in listOfCoordinates:
            newrow=np.zeros(n*n*n)
            newrow[to1D(e[0],e[1],k-1,size)]=1
            A=np.vstack((A,newrow))
    return(A)

In [86]:
## solving
def solving(A):
    b=matrix(np.ones(A.shape[0])) 
    c=matrix(np.zeros(A.shape[1])) 
    G=matrix(np.zeros(A.shape))
    h=matrix(np.zeros(A.shape[0]))
    A1 = matrix(A)
    (status, solution) = cvxopt.glpk.ilp(c=c,G=G,h=h,A=A1,b=b,B=set(range(A.shape[1])))
    return (status, solution)

In [87]:
# The function that transforms the solution from the binary form to a square matrix with the right number in each cell.
def to2D(a,size=3):
    n=size**2
    E=np.zeros((n,n),dtype=int)
    for i in range(n):
        for j in range(n):
            for k in range(n):
                if a[to1D(i,j,k,size)]==1:
                    E[i,j]=k+1
    return(E)

In [88]:
#The function that prints the solution in the form of a sudoku grid.
def print_sudoku(grid,size=3):
    for r in range(len(grid)):
        row = ""
        for c in range(len(grid[r])):
            if c%size == 0:
                row += "["
            row += " "+str(grid[r][c])
            if c%size == size-1:
                row += " ]"
        print(row)
        if r % size == size-1:
            print("-"*27)

In [103]:
#The function that solves the sudoku puzzle, it takes as parameters: B the initial grid with the known numbers, and the size of
#the puzzle, in our case size=3
def sudoku(B,size=3):
    A=const(B,size)
    solution=solving(A)[1]
    E=to2D(solution,size)
    print_sudoku(E,size)

## 3 - Test
- Simple Sudoku Puzzle

In [104]:
B = np.array([
[0,8,0, 9,0,1, 0,5,0],
[0,0,2, 6,8,7, 3,0,0],
[0,0,3, 0,0,0, 6,0,0],
[3,9,0, 0,0,0, 0,6,5],
[6,0,0, 4,7,5, 0,0,3],
[5,7,0, 0,0,0, 0,8,4],
[0,0,9, 0,0,0, 8,0,0],
[0,0,5, 1,2,4, 9,0,0],
[0,4,0, 8,0,3, 0,2,0]])
print_sudoku(B,3)

[ 0 8 0 ][ 9 0 1 ][ 0 5 0 ]
[ 0 0 2 ][ 6 8 7 ][ 3 0 0 ]
[ 0 0 3 ][ 0 0 0 ][ 6 0 0 ]
---------------------------
[ 3 9 0 ][ 0 0 0 ][ 0 6 5 ]
[ 6 0 0 ][ 4 7 5 ][ 0 0 3 ]
[ 5 7 0 ][ 0 0 0 ][ 0 8 4 ]
---------------------------
[ 0 0 9 ][ 0 0 0 ][ 8 0 0 ]
[ 0 0 5 ][ 1 2 4 ][ 9 0 0 ]
[ 0 4 0 ][ 8 0 3 ][ 0 2 0 ]
---------------------------


In [105]:
sudoku(B,3)

[ 7 8 6 ][ 9 3 1 ][ 4 5 2 ]
[ 4 5 2 ][ 6 8 7 ][ 3 1 9 ]
[ 9 1 3 ][ 5 4 2 ][ 6 7 8 ]
---------------------------
[ 3 9 4 ][ 2 1 8 ][ 7 6 5 ]
[ 6 2 8 ][ 4 7 5 ][ 1 9 3 ]
[ 5 7 1 ][ 3 6 9 ][ 2 8 4 ]
---------------------------
[ 2 3 9 ][ 7 5 6 ][ 8 4 1 ]
[ 8 6 5 ][ 1 2 4 ][ 9 3 7 ]
[ 1 4 7 ][ 8 9 3 ][ 5 2 6 ]
---------------------------


- Very hard Sudoku Puzzle

In [106]:
C = np.array([
[7,0,0, 0,0,0, 4,0,0],
[0,2,0, 0,7,0, 0,8,0],
[0,0,3, 0,0,8, 0,0,9],
[0,0,0, 5,0,0, 3,0,0],
[0,6,0, 0,2,0, 0,9,0],
[0,0,1, 0,0,7, 0,0,6],
[0,0,0, 3,0,0, 9,0,0],
[0,3,0, 0,4,0, 0,6,0],
[0,0,9, 0,0,1, 0,0,5]])

print_sudoku(C,3)

[ 7 0 0 ][ 0 0 0 ][ 4 0 0 ]
[ 0 2 0 ][ 0 7 0 ][ 0 8 0 ]
[ 0 0 3 ][ 0 0 8 ][ 0 0 9 ]
---------------------------
[ 0 0 0 ][ 5 0 0 ][ 3 0 0 ]
[ 0 6 0 ][ 0 2 0 ][ 0 9 0 ]
[ 0 0 1 ][ 0 0 7 ][ 0 0 6 ]
---------------------------
[ 0 0 0 ][ 3 0 0 ][ 9 0 0 ]
[ 0 3 0 ][ 0 4 0 ][ 0 6 0 ]
[ 0 0 9 ][ 0 0 1 ][ 0 0 5 ]
---------------------------


In [107]:
sudoku(C,3)

[ 7 9 8 ][ 6 3 5 ][ 4 2 1 ]
[ 1 2 6 ][ 9 7 4 ][ 5 8 3 ]
[ 4 5 3 ][ 2 1 8 ][ 6 7 9 ]
---------------------------
[ 9 7 2 ][ 5 8 6 ][ 3 1 4 ]
[ 5 6 4 ][ 1 2 3 ][ 8 9 7 ]
[ 3 8 1 ][ 4 9 7 ][ 2 5 6 ]
---------------------------
[ 6 1 7 ][ 3 5 2 ][ 9 4 8 ]
[ 8 3 5 ][ 7 4 9 ][ 1 6 2 ]
[ 2 4 9 ][ 8 6 1 ][ 7 3 5 ]
---------------------------


In [108]:
D = np.array([
[0,2, 4,0],
[0,0, 0,2],
[3,0, 0,0],
[0,1, 3,0]])

print_sudoku(D,2)

[ 0 2 ][ 4 0 ]
[ 0 0 ][ 0 2 ]
---------------------------
[ 3 0 ][ 0 0 ]
[ 0 1 ][ 3 0 ]
---------------------------


In [109]:
sudoku(D,2)

[ 1 2 ][ 4 3 ]
[ 4 3 ][ 1 2 ]
---------------------------
[ 3 4 ][ 2 1 ]
[ 2 1 ][ 3 4 ]
---------------------------
